In [1]:
import pandas as pd
import numpy as np
import datetime
import tqdm

In [3]:
data_train = pd.read_csv('../data/coms_sep/train.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
import os
import re
FIRST_N = 100
N_THREADS = 5
test_folder = '../data/raw_splits/test/'
test_files = sorted([x for x in os.listdir(test_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
test_files = [os.path.join(test_folder, x) for x in test_files]
print(f'Length of test files is {len(test_files)}')
train_folder = '../data/raw_splits/train/'
train_files = sorted([x for x in os.listdir(train_folder) if not '.pkl' in x], key = lambda x: int(re.sub('[^0-9]', '', x)))
train_files = [os.path.join(train_folder, x) for x in train_files]
print(f'Length of train files is {len(train_files)}')

Length of test files is 26
Length of train files is 51


['../data/raw_splits/train/chunk_0.fth',
 '../data/raw_splits/train/chunk_1.fth',
 '../data/raw_splits/train/chunk_2.fth',
 '../data/raw_splits/train/chunk_3.fth',
 '../data/raw_splits/train/chunk_4.fth']

In [7]:
cat_feats = ['cat_new_ip', 'cat_new_prov', 'op_type', 'relative', 'cdf_s_127', 'cdf_s_135',
             'cdf_s_130', 'cdf_s_129', 'cdf_s_134', 'cdf_s_133', 'know_recip_card_age', 'one_region']
num_feats = ['amount', 'client_age', 'age_diff', 'cumulative_sum_total', 'cumulative_sum_total', 'data_i_120',
             'know_recip_power', 'data_i_120', 'recip_card_age', 'krp_pow2', 'log_amount']
feats = cat_feats + num_feats

In [8]:
target = data_train['label']
features = data_train[feats]

In [11]:
# как уже говорились при работе с данными, имеющими временной аспект крайне важно отсекать транзакции,
# идущие после оцениваемого события

# кроме того зачастую бывает полезно создавать фичи не за весь доступный горизонт данных,
# а за различные временные окна


def timeframe_trans(
    trans, 
    date_threshold=None,
    depth=None):
    """
    Функция отбирающая тразакции в заданном временном окне
    :param trans: весь массив транзакций
    :param date_threshold: дата и время отсечки, если не указан, то datetime.now()
    :param depth: глубина периода от даты отсечки, если не указан, то от текущего момента
    :return:

    """
    if date_threshold and depth:
        interested_trans = trans[trans.event_time.between(
            date_threshold - depth, date_threshold)]
    elif date_threshold and depth is None:
        interested_trans = trans[trans.event_time <= date_threshold]
    elif date_threshold is None and depth:
        interested_trans = trans[trans.event_time.between(
            datetime.datetime.now() - depth, datetime.datetime.now())]
    else:
        interested_trans = trans

    return interested_trans


def calc_quants_per_user(
    trans, 
    col_name='amount_original', 
    quants=np.arange(0.1, 1.1, .1), 
    feat_name='amount_q_', 
    depth=None, 
    start_date=datetime.date(2017, 10, 29),
    end_date=datetime.date(2017, 11, 29)):
    """
    Функция рассчитывающая квантили для пользователя с округлением до дня
    """
    
    time_delta = end_date - start_date
    # можно переопределить и выбрать другой временной интервал
    list_of_days = [
        start_date + datetime.timedelta(days=i) for i in range(time_delta.days + 1)]

    rez_df = pd.DataFrame()

    for day in list_of_days:
        if depth is None:
            cur_cutoff = trans[trans.event_time < day]
        else:
            cur_cutoff = trans[(trans.event_time < day) & (
                trans.event_time > (day - depth))]

        if cur_cutoff.shape[0] >= 1:
            res = cur_cutoff[col_name].quantile(quants)
        else:
            res = [0 for i in quants]
        for i in zip(quants, res):
            rez_df.loc[day, feat_name + str(i[0])] = i[1]

    return rez_df


def all_usr_quants(
    users_list,
    chunks_names,
    number_of_splits=900,
    col_name='amount_original',
    quants=np.arange(0.1, 1.1, .1),
    feat_name='amount_q_',
    depth=None,
    start_date=datetime.date(2017, 10, 29),
    end_date=datetime.date(2017, 11, 29)):
    """
    Расчет квантилей по всем пользователям со сплитованием на подмножества
    """
    rez_df = pd.DataFrame()
    usr_chunks = np.array_split(users_list, number_of_splits)

    for chunk in tqdm.tqdm_notebook(usr_chunks):
        trans_data = load_data(chunks_names, fields = ['user_id', 'event_time', col_name],
                               query = f"user_id in ({str(users_list)[1:-1]})")
#         trans_data = (df_from_sql("select user_id, event_time, {col} from rsa_event_log where user_id in ({users})"
#                                   .format(col = col_name, users=str(list(chunk))[1:-1]), parse_dates=['event_time']))
        chunk_df =  trans_data.groupby('user_id').apply(lambda x: 
                        # В даннос случае x - dataframe с транзакциями 1ого пользователя
                        calc_quants_per_user(x, col_name=col_name, feat_name=feat_name, quants=quants, depth = depth, start_date=start_date, end_date=end_date)
                    )
        rez_df = pd.concat([rez_df, chunk_df])
        
    return rez_df

In [5]:
import numpy as np
import pandas as pd
# import datetime


def transform_cols (df, dict_col_types = None):
    # Расширяйте для необходимых столбцов и их явной типизации
    if dict_col_types is None:
        dict_col_types = {
        'amount_original':(float, 0.0),
        'channel_indicator_desc':(str, u'null'),
        'event_description':(str, u'null'),
        'short_date':(int, 0),
        'cdf_s_20':(str, u'null'),
        'cdf_s_126':(str, u'null'),
        'cdf_s_127':(int, 30),
        'cdf_s_129':(int, 30),
        'cdf_s_138':(str, u'null'),
        'cdf_s_130':(int, 30),
        'cdf_s_133':(int, 30),
        'cdf_s_134':(int, 30),
        'cdf_s_135':(int, 30),
        'cdf_s_140':(float, 0.0),
        'cdf_s_218':(str, u'null'),
        'cdf_s_294':(int, 0),
        'cdf_s_299':(str, u'null'),
        'data_s_65':(int, 0),
        'data_i_120':(int, 0),
        'data_i_154':(float, -150)
        }
                
    if df.shape[0] == 0:
        return df
    
    df.replace(u'null', np.nan, inplace=True)

    for i in dict_col_types:
        if i in df.columns:
            change_type, fill_value = dict_col_types[i]
            df[i] = df[i].fillna(fill_value).astype(change_type)
    
    return df

def load_data(chunk_fnames, fields=None, query=None, sample='train', dict_col_types=None):
    df = pd.DataFrame({})
    if isinstance(chunk_fnames, str):
        chunk_fnames = [chunk_fnames]
        
    for filename in tqdm.tqdm_notebook(chunk_fnames):
        chunk_df = pd.read_feather(filename)
            
        if fields is None:
            fields = chunk_df.columns.tolist()
        
        transormed = transform_cols(chunk_df)
        
        if query:
            transormed = transormed.query(query)
 
        df = pd.concat([df, transormed[fields]], ignore_index=True)
    return df

In [166]:
def get_data(chunk_name):
    chunk_df = pd.read_feather(chunk_name)[['user_id', 'event_time',
                                           'amount_original',
                                           'channel_indicator_desc']]
    return chunk_df

In [172]:
import multiprocessing

In [175]:
%%time
# create features matrix
with multiprocessing.Pool(processes=min(1, FIRST_N)) as pool:
    results = pool.map(get_data, train_files)

CPU times: user 28 s, sys: 16 s, total: 44 s
Wall time: 7min 28s


In [177]:
quantile_df = pd.DataFrame()
for i, df in enumerate(results):
    quantile_df = pd.concat([quantile_df, df])

In [182]:
quantile_df = quantile_df[quantile_df['amount_original'].values != "null"]

In [185]:
quantile_df[:5]

,user_id,event_time,amount_original,channel_indicator_desc
0,100197,2017-08-25 22:47:42,10000.0,MOBILEAPI
2,100197,2017-05-07 09:46:44,1600.0,MOBILEAPI
4,100197,2017-09-13 14:30:56,18200.0,MOBILEAPI
5,100197,2017-10-07 16:14:31,568.0,MOBILEAPI
6,100197,2017-08-03 10:46:32,8900.0,MOBILEAPI


In [2]:
# quantile_df.to_csv("../data/coms_sep/for_quantiles.csv")
prequants = pd.read_csv("../data/coms_sep/for_quantiles.csv", index_col=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
prequants[:5]

,user_id,event_time,amount_original,channel_indicator_desc
0,100197,2017-08-25 22:47:42,10000.0,MOBILEAPI
2,100197,2017-05-07 09:46:44,1600.0,MOBILEAPI
4,100197,2017-09-13 14:30:56,18200.0,MOBILEAPI
5,100197,2017-10-07 16:14:31,568.0,MOBILEAPI
6,100197,2017-08-03 10:46:32,8900.0,MOBILEAPI


In [18]:
frstrow = prequants.iloc[0, :]

In [19]:
datetime.tfrstrow[1]

'2017-08-25 22:47:42'

In [ ]:
def mini_quants(x):
    out = calc_quants_per_user(
        x,
        col_name=col_name,
        feat_name=feat_name,
        quants=quants,
        depth = depth,
        start_date=start_date,
        end_date=end_date)

In [20]:
prequants['event_time'] = pd.to_datetime(prequants['event_time'])

In [7]:
prequants[prequants['user_id'] == 100197].head(5)

,user_id,event_time,amount_original,channel_indicator_desc
0,100197,2017-08-25 22:47:42,10000.0,MOBILEAPI
2,100197,2017-05-07 09:46:44,1600.0,MOBILEAPI
4,100197,2017-09-13 14:30:56,18200.0,MOBILEAPI
5,100197,2017-10-07 16:14:31,568.0,MOBILEAPI
6,100197,2017-08-03 10:46:32,8900.0,MOBILEAPI


In [26]:
amen = calc_quants_per_user(
    trans=prequants.groupby("user_id"),
    col_name="amount_original",
    depth=datetime.timedelta(days=30),
    start_date=datetime.date(2017, 10, 29),
    end_date=datetime.date(2017, 11, 29))

TypeError: '<' not supported between instances of 'SeriesGroupBy' and 'datetime.date'

In [22]:
amen

,amount_q_0.1,amount_q_0.2,amount_q_0.30000000000000004,amount_q_0.4,amount_q_0.5,amount_q_0.6,amount_q_0.7000000000000001,amount_q_0.8,amount_q_0.9,amount_q_1.0
2017-10-29,70.0,100.0,160.0,313.6,1650.0,6560.0,10240.0,11440.0,21800.0,40000.0
2017-10-30,70.0,100.0,115.0,250.0,1109.0,4640.0,9720.0,11660.0,21800.0,40000.0
2017-10-31,70.0,100.0,115.0,250.0,1109.0,4640.0,9720.0,11660.0,21800.0,40000.0
2017-11-01,70.0,100.0,160.0,313.6,1650.0,3600.0,8880.0,11440.0,21800.0,40000.0
2017-11-02,160.0,250.0,784.4,1860.0,4000.0,6420.0,10240.0,14040.0,21800.0,40000.0
2017-11-03,160.0,250.0,784.4,1860.0,4000.0,6420.0,10240.0,14040.0,21800.0,40000.0
2017-11-04,160.0,250.0,784.4,1860.0,4000.0,6420.0,10240.0,14040.0,21800.0,40000.0
2017-11-05,175.0,250.0,784.0,1650.0,3000.0,5900.0,9100.0,12100.0,21800.0,40000.0
2017-11-06,145.0,250.0,536.2,1130.0,1825.0,3600.0,6030.0,8720.0,11770.0,40000.0
2017-11-07,115.0,250.0,475.0,1260.0,1825.0,3200.0,5330.0,6940.0,10620.0,40000.0


In [ ]:
amen = calc_quants_per_user(trans=)